In [67]:
import pandas as pd
import numpy as np

In [68]:
df = pd.read_csv('Expanded_data_with_more_features.csv', index_col=0)
df.head()

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75


##   1. Скільки студентів і студенток у наборі даних?

In [69]:
len(df[df.Gender == 'female'])

15424

In [70]:
len(df[df.Gender == 'male'])

15217

## 2. Який відсоток студентів чоловічої статі в наборі даних?

In [71]:
len(df[df.Gender == 'male']) / len(df) * 100

49.6622172905584

## 3. Який відсоток студенток у наборі даних?

In [72]:
len(df[df.Gender == 'female']) / len(df) * 100

50.337782709441605

## 4. Яка етнічна група має найвищий середній бал з математики?

In [73]:
df.groupby('EthnicGroup')['MathScore'].mean().sort_values(ascending=False)[:1]

EthnicGroup
group E    75.298936
Name: MathScore, dtype: float64

## 5. Яка етнічна група має найвищий середній бал читання?

In [74]:
df.groupby('EthnicGroup')['ReadingScore'].mean().sort_values(ascending=False)[:1]

EthnicGroup
group E    74.251423
Name: ReadingScore, dtype: float64

## 6. Яка етнічна група має найвищий середній бал з письма? 

In [75]:
df.groupby('EthnicGroup')['WritingScore'].mean().sort_values(ascending=False)[:1]

EthnicGroup
group E    72.67706
Name: WritingScore, dtype: float64

## 7. Який середній бал з математики для студентів, які закінчили курс підготовки до тесту?

In [76]:
df[df['TestPrep'] == 'completed']['MathScore'].mean()

69.5466599698644

## 8. Який середній бал з читання для студентів, які пройшли курс підготовки до тесту?

In [77]:
df[df['TestPrep'] == 'completed']['ReadingScore'].mean()

73.73299849321948

## 9. Який середній бал для студентів, які закінчили курс підготовки до тесту?

In [78]:
df['MeanScore'] = df[['MathScore', 'ReadingScore', 'WritingScore']].mean(axis=1)
df[df['TestPrep'] == 'completed']['MeanScore'].mean()

72.66097438473135

## 10. Скільки студентів мають одружених батьків?

In [79]:
len(df[df['ParentMaritalStatus'] == 'married'])

16844

## 11. Скільки студентів мають батьків-одинаків?

In [80]:
len(df[df['ParentMaritalStatus'] == 'single'])

7097

## 12. Чи студенти, які регулярно займаються спортом, мають вищі середні результати з математики, ніж ті, хто ніколи не займається спортом?

In [81]:
df[df['PracticeSport'] == 'regularly']['MathScore'].mean() > df[df['PracticeSport'] == 'never']['MathScore'].mean()

True

## 13. Чи студенти, які регулярно займаються спортом, мають вищі середні результати читання, ніж ті, хто ніколи не займається спортом?

In [82]:
df[df['PracticeSport'] == 'regularly']['ReadingScore'].mean() > df[df['PracticeSport'] == 'never']['ReadingScore'].mean()

True

## 14. Скільки учнів користується шкільним автобусом, щоб дістатися до школи?

In [83]:
len(df[df['TransportMeans'] == 'school_bus'])

16145

## 15. Скільки учнів користується приватним транспортом, щоб дістатися до школи?

In [84]:
len(df[df['TransportMeans'] == 'private'])

11362

In [85]:
from sklearn.preprocessing import StandardScaler
import sklearn as sk
import matplotlib.pyplot as plt

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

In [86]:
num_features = [
    'MathScore', 
    'ReadingScore', 
    'WritingScore',
    'NrSiblings'
]
cat_features = [
    'Gender', 
    'EthnicGroup', 
    'ParentEduc', 
    'LunchType', 
    'ParentMaritalStatus', 
    'PracticeSport',
    'IsFirstChild',
    'TransportMeans',
    'WklyStudyHours'
]

In [87]:
df['IsTestCompleted'] = df['TestPrep'] == 'completed'

In [88]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),
    ('scaler', StandardScaler())
])

pipeline = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", OneHotEncoder(), cat_features)
])

X = pipeline.fit_transform(df)
Y = df['IsTestCompleted'].values

In [89]:
model = HistGradientBoostingClassifier()
cv = ShuffleSplit(n_splits=15, test_size=0.25, random_state=0)
cross_val_score(HistGradientBoostingClassifier(), X, Y, cv=cv).mean()

0.7454118261323586